In [ ]:
import calliope

# Loading a model and inspecting inputs

In [ ]:
# The `model.yaml` file will import all other yaml files defined within it.
# Another approach is to define all information about the model in a single file (locations, techs, etc.), 
# but this can get quite lengthy!

model = calliope.Model("model-2/model.yaml")

In [ ]:
# Loaded data is stored as an xarray dataset, which can be saved as a single NetCDF.
# In this model, we've defined data given the units
# GW/GWh for power/energy, Million Euros for monetary costs, and km^2 for land area
model.inputs

In [ ]:
# model data is stored using concatenated sets, to cut down on largely empty arrays. 
# To handle the data yourself, you will probably prefer to have the sets un-concatenated
model.get_formatted_array("energy_cap_max")

In [ ]:
# For small arrays, you can also view the data more easily using Pandas series
model.get_formatted_array("energy_cap_max").to_series().dropna()

## Plotting data

In [ ]:
# Inputs can be plotted using internal plotting functionality...
model.plot.timeseries()

In [ ]:
# ... but you might like to plot something yourself
ax = model.get_formatted_array("lifetime").to_pandas().plot.bar()
ax.legend(bbox_to_anchor=(1, 1))
ax.set_ylabel("Technology lifetime (years)")

# Running the model

In [ ]:
model.run()

In [ ]:
# Looking at the timeseries, we see there is one day with some unmet electricity demand.
model.plot.timeseries()

In [ ]:
model.get_formatted_array("energy_cap").to_series().dropna().round(2)

# Run with scenarios

## Add transmission line limits

In [ ]:
# The current model has no limits on transmission line capacity. 
# This may not be realistic, so let's try adding some line limits in
model2 = calliope.Model("model-2/model.yaml", scenario="add-transmission-limits")

In [ ]:
model2.run()

# Add a medium-term storage option

In [ ]:
# Maybe storage can help?
model2 = calliope.Model("model-2/model.yaml", scenario="add-transmission-limits,add-storage")

In [ ]:
model2.run()

In [ ]:
model2.plot.timeseries()

In [ ]:
model2.plot.capacity()

# Comparing model outputs

In [ ]:
import pandas as pd

In [ ]:
# Let's look at system costs and capacities between our two models
costs = pd.Series(
    [model.get_formatted_array("cost").sel(costs="monetary").to_series().sum(),
     model2.get_formatted_array("cost").sel(costs="monetary").to_series().sum()],
    index=["First model run", "Second model run"]
)
capacities = pd.concat(
    [model.get_formatted_array("energy_cap").sum("locs").to_series(),
     model2.get_formatted_array("energy_cap").sum("locs").to_series()],
    keys=["First model run", "Second model run"], axis=1
)

# Transmission is a bit of a strange one to handle at the moment.
# This code snippet will sum up all transmission links and then half the result 
# (since the sum includes the capacity in both directions)
capacities = capacities.groupby(lambda x: x.split(":")[0], axis=0).sum()
capacities.loc["ac_transmission"] = capacities.loc["ac_transmission"].div(2)

In [ ]:
costs

In [ ]:
colors = model.inputs.colors.to_series().to_dict()
ax = capacities.T.plot.bar(stacked=True, color=colors)
ax.legend(bbox_to_anchor=(1, 1))